In [1]:
from PIL import Image
from numpy import array
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


In [2]:
data = np.load("zdmdtImages.npz")
print data['Y'][:-3].shape,data['X'].shape

batch_size = 32
num_classes = 2
epochs = 12
# splitting data into train and test
x_train, x_test, y_train, y_test= train_test_split(data['X'], data['Y'][:-3], test_size=0.20, random_state=42, stratify=data['Y'][:-3])
# splitting train into valid dataset
print x_train.shape
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.30, random_state=42, stratify=y_train)
print x_train.shape
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape: ', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(35425,) (35425, 22, 24, 4)
(28340, 22, 24, 4)
(19838, 22, 24, 4)
('x_train shape: ', (19838, 22, 24, 4))
(19838, 'train samples')
(7085, 'test samples')


In [3]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [24]:
# with regularisation in dense layer
model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(22, 24, 4)))
model.add(Conv2D(16, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(2048, activation='relu', kernel_regularizer=regularizers.l2(0.02)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adagrad(),
              metrics=['accuracy'])
# adaptive gradient, batch size should increase, avoid overfitting, increase epochs
class_weight = {0: 1.,
                1: 7.}

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 20, 22, 8)         296       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 18, 16)        3216      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 9, 16)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 8, 9, 16)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 2048)              2361344   
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
__________

In [25]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=1e-6,patience=5,verbose=0, mode='auto')

model.fit(x_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=1,
          validation_data=(x_valid,y_valid),
          callbacks=[earlyStopping],
         class_weight=class_weight)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 19838 samples, validate on 8502 samples
Epoch 1/40
19838/19838 [==============================] - 51s 3ms/step - loss: 2.0961 - acc: 0.5324 - val_loss: 13.3667 - val_acc: 0.1665
Epoch 2/40
19838/19838 [==============================] - 48s 2ms/step - loss: 1.0671 - acc: 0.7482 - val_loss: 3.7338 - val_acc: 0.7483
Epoch 3/40
19838/19838 [==============================] - 47s 2ms/step - loss: 1.0182 - acc: 0.7553 - val_loss: 7.6073 - val_acc: 0.5134
Epoch 4/40
19838/19838 [==============================] - 46s 2ms/step - loss: 0.9881 - acc: 0.7622 - val_loss: 7.1359 - val_acc: 0.5300
Epoch 5/40
19838/19838 [==============================] - 48s 2ms/step - loss: 0.9735 - acc: 0.7662 - val_loss: 7.9681 - val_acc: 0.4812
Epoch 6/40
19838/19838 [==============================] - 47s 2ms/step - loss: 0.9492 - acc: 0.7692 - val_loss: 6.6426 - val_acc: 0.5652
Epoch 7/40
19838/19838 [==============================] - 52s 3ms/step - loss: 0.9386 - acc: 0.7746 - val_loss: 5.2187 - val_acc

In [26]:
ypred = model.predict(x_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test.argmax(axis=1),ypred.argmax(axis=1))

array([[5281,  868],
       [ 306,  630]])

In [28]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test.argmax(axis=1), ypred.argmax(axis=1))

(array([ 0.94523   ,  0.42056075]),
 array([ 0.85883884,  0.67307692]),
 array([ 0.89996592,  0.51766639]),
 array([6149,  936]))